In [ ]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = 'mypassword'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Dynamic Pricing Engine

## Description of the use case
A dynamic pricing engine adjusts the prices of products in real-time based on various factors like demand, competition, and inventory levels. This use case ensures that pricing strategies are optimized to maximize revenue and profitability.

## Redis Data Structures Used
- **Sorted Sets**: To manage and query price changes and demand metrics.
- **Hashes**: To store product pricing information and other related attributes.

In [ ]:
import random

# Mock data for the use case
mock_data = {
    'product_1': {'name': 'Product 1', 'price': 100},
    'product_2': {'name': 'Product 2', 'price': 150},
    'product_3': {'name': 'Product 3', 'price': 200},
}

# Use Hashes to store product pricing information
for product_id, product_info in mock_data.items():
    r.hset(product_id, mapping=product_info)
    print(f"Old Product Info : {product_id} : {product_info}")

# Use Sorted Sets to manage price changes and demand metrics
for product_id, product_info in mock_data.items():
    r.zadd('product_prices', {product_id: product_info['price']})
    r.zadd('product_demand', {product_id: random.randint(10,100)}) # Random demand

# Dynamic pricing algorithm
def dynamic_pricing_engine(product_id, base_price):
    demand = int(r.zscore('product_demand', product_id))
    new_price = base_price + (demand * 0.1)  # Example algorithm: price increases by 10% of demand
    r.hset(product_id, 'price', new_price)
    r.zadd('product_prices', {product_id: new_price})
    return new_price

# Update prices for all products
for product_id in mock_data.keys():
    base_price = int(r.hget(product_id, 'price'))
    new_price = dynamic_pricing_engine(product_id, base_price)
    print(f"Updated price for {product_id}: {new_price}")

# Print the updated pricing information
for product_id in mock_data.keys():
    print(f"New Product Info :{product_id}: {r.hgetall(product_id)}")


In [ ]:
r.zrange('product_prices',0,-1, withscores=True)

In [ ]:
r.zrange('product_demand', 0, -1, withscores=True)

In [ ]:
for item in r.scan_iter("product*", _type='HASH'):
    print(r.hgetall(item))